In [22]:
# Imports
import pandas as pd
import numpy as np
import pickle

In [23]:
# Read Diagnostic Data
data = pd.read_csv('ADNIMERGE.csv')

In [24]:
# We only need ID, Exam Data, Baseline, and Current
dx_data = data[['PTID', 'EXAMDATE', 'DX_bl', 'DX']]

In [25]:
# We only want Diagnoses which have changed
m = dx_data.query('DX != DX_bl')

In [26]:
# Convert to numpy
arr = m.to_numpy()

In [27]:
def subject_to_int(val):
    return(int(val[0][0:3]) * 1000 + int(val[0][6:10]))

In [28]:
arr = sorted(arr, key=subject_to_int)

In [29]:
# If we changed from MCI to AD
last = ''
count = 0
arr_s = []
for val in arr:
    if val[0] == last:
        count += 1
    else:
        count = 0
        last = val[0]
    if val[2] == 'LMCI' and val[3] == 'Dementia':
        # if this is the case, scan backwards to figure out how many images we have before and add that to val
        tmp = list(val)
        tmp.append(count)
        arr_s.append(tmp)

In [30]:
# only capture the first instance, so the breakthrough Diagnosis
arr_final_AD = []
last_names = set()
for val in arr_s:
    if val[0] not in last_names:
        arr_final_AD.append(list(val))
        last_names.add(val[0])   

In [31]:
# measure how many previous visits each patient has
count_arr = np.zeros(100)
for val in arr_final_AD:
    count_arr[val[4]] += 1
    
    

In [32]:
# no need to include redundant data
for i in range(len(arr_final_AD)):
    arr_final_AD[i] = [arr_final_AD[i][0], arr_final_AD[i][1], arr_final_AD[i][4]]

In [33]:
# Save it as a csv
pd.DataFrame(arr_final_AD).to_csv("./MCI_to_AD.csv")

In [34]:
# capture all patients who started as MCI and remained as MCI
# take all patients who started as MCI, and subtract arr_f from it
MCI_arr = dx_data.to_numpy()
arr_m = []
for val in MCI_arr:
    if val[2] == 'LMCI':
        arr_m.append(list(val))

In [35]:
arr_m = sorted(arr_m, key=subject_to_int)

In [9]:
# go through each value
# if we get dementia or nan, skip everything on the current ID, and do not append
# otherwise, when we change, append

# x is whatever nan is in the original array. Super hacky, but whatever
x = arr_m[4983][3]

arr_final_MCI = []
last_names = set()
last_val = ''
count = 0
append = True
# go through each entry. When the subject ID changes (or is at an end), if there were no instances of dementia, 
# append that patient ID and the count
for val in arr_m:
    if val[0] not in last_names:
        if append:
            tmp = list(last_val)
            tmp.append(count)
            arr_final_MCI.append(tmp)
        last_names.add(val[0])
        count = 0
        append = True
    if val[3] in ['Dementia']:
        append = False
    last_val = val
    count += 1
    

In [18]:
arr_final_MCI = arr_final_MCI[1::]

In [19]:
# measure how many previous visits each patient has
count_arr = np.zeros(30)
for val in arr_final_MCI:
    count_arr[val[4]] += 1
    
print(count_arr)

[ 0. 54. 62. 20. 18. 24. 23. 39. 24. 10.  7. 14.  5.  8.  7.  7.  6.  6.
  4.  4.  2.  4.  0.  0.  0.  0.  0.  0.  0.  0.]


In [20]:
# no need to include redundant data
for i in range(len(arr_final_MCI)):
    arr_final_MCI[i] = [arr_final_MCI[i][0], arr_final_MCI[i][1], arr_final_MCI[i][4]]

In [21]:
pd.DataFrame(arr_final_MCI).to_csv("./MCI_to_MCI.csv")